In [2]:
import pandas as pd
import osmnx as ox
import matplotlib.pyplot as plt
import json
from sqlalchemy.orm import subqueryload
from sqlalchemy_bigquery import WKT
from shapely.geometry import Point
from sqlalchemy import select, func, Boolean, text
from geoalchemy2.shape import from_shape, to_shape
import shapely
import numpy

from src.database.models import *
from src.database.base import *

In [4]:
# bound = {"x1": 56.222, "y1": 43.894, "x2": 56.429, "y2": 44.086}
session = AsyncSessionMaker()
cnt = 20000
last_city = None
pops = []
# df = pd.read_csv("file.csv", nrows=cnt)
# print(df)

SIZE = 0.0083333
M = 1


def create_poly(poly, size: int):
    left_bottom = (poly.x - size * M / 2, poly.y - size / 2)
    left_top = (left_bottom[0], left_bottom[1] + size)
    right_top = (left_top[0] + size * M, left_top[1])
    right_bottom = (right_top[0], right_top[1] - size)
    coords = (left_bottom, left_top, right_top, right_bottom)
    return shapely.Polygon(coords)


async def get_city(p):
    stmt = (
        select(City).filter(func.ST_Intersects(City.geometry, from_shape(p))).limit(1)
    )
    result = await session.execute(stmt)
    return result.scalar_one_or_none()


async def in_bound(p):
    c = None
    global last_city
    if last_city:
        g = to_shape(last_city.geometry)
        if g.contains(p):
            c = last_city
        else:
            city = await get_city(p)
            if city:
                last_city = city
                c = last_city

    else:
        city = await get_city(p)
        if city:
            last_city = city
            c = last_city
    return c


iter = 0
for chunk in pd.read_csv("ukr_file.csv", chunksize=cnt):
    iter += 1
    for c in chunk.itertuples():
        if c.Z >= 3:
            p = Point(c.X, c.Y)
            ci = await in_bound(p)
            if ci:
                location = WKT(p.wkt)
                poly = create_poly(p, SIZE)
                geometry = WKT(poly.wkt)
                pop = Population(
                    location=location, geometry=geometry, value=c.Z, city=ci
                )
                pops.append(pop)
    print(iter * cnt, len(pops))

session.add_all(pops)
await session.commit()

20000 0
40000 0
60000 0
80000 0
100000 11
120000 11
140000 11
160000 11
180000 11
200000 11
220000 11
240000 15
260000 16
280000 16
300000 16
320000 16
340000 44
360000 47
380000 48
400000 48
420000 59
440000 150
460000 203
480000 230
500000 272
520000 302
540000 310
560000 459
580000 612
600000 812
620000 867
640000 1042
660000 1499
680000 1949
700000 2182
720000 2427
740000 2838
760000 2953
780000 3506
800000 4439
820000 5095
840000 5391
860000 5497
880000 5751
900000 6064
920000 6457
940000 6536
960000 7067
980000 7271
1000000 7380
1020000 9419
1040000 11554
1060000 14832


In [ ]:
# import math

SIZE = 0.0083333
M = 1


def create_poly(poly: dict, size: int):
    left_bottom = (poly.x - size * M / 2, poly.y - size / 2)
    left_top = (left_bottom[0], left_bottom[1] + size)
    right_top = (left_top[0] + size * M, left_top[1])
    right_bottom = (right_top[0], right_top[1] - size)
    coords = (left_bottom, left_top, right_top, right_bottom)
    return shapely.Polygon(coords)


session = AsyncSessionMaker()
result = await session.execute(select(City))
cities = result.scalars()


for city in cities:
    result = await session.execute(
        select(Population).where(Population.city_id == city.id)
    )
    populations = result.scalars()
    populations = [p for p in populations]
    if len(populations) == 0:
        continue

    for population in populations:
        p = to_shape(population.location)
        poly = create_poly(p, SIZE)
        geometry = WKT(poly.wkt)
        population.geometry = geometry

    session.add_all(populations)
    await session.commit()
    print(city.name)
    # min_r = 1234567890000123456789345678
    # for i in range(l_p):
    #     p_i = to_shape(populations[i].location)
    #     for j in range(i + 1, l_p):
    #         p_j = to_shape(populations[j].location)
    #         r = math.hypot(p_i.x - p_j.x, p_i.y - p_j.y)
    #         if r < min_r:
    #             min_r = r
    # print(min_r)

In [6]:
session = AsyncSessionMaker()

with open("parking_NN.json", encoding="utf-8") as file:
    data = json.load(file)
parkings = data["features"]

parks = []
for p in parkings:
    prop = p["properties"]
    location = WKT(
        Point(p["geometry"]["coordinates"][0], p["geometry"]["coordinates"][1]).wkt
    )

    left_bottom = (prop["boundedBy"][0][0], prop["boundedBy"][0][1])
    left_top = (prop["boundedBy"][0][0], prop["boundedBy"][1][1])
    right_top = (prop["boundedBy"][1][0], prop["boundedBy"][1][1])
    right_bottom = (prop["boundedBy"][1][0], prop["boundedBy"][0][1])
    coords = (left_bottom, left_top, right_top, right_bottom)
    park = Parking(
        description=prop["description"],
        address=prop["CompanyMetaData"]["address"],
        location=location,
        city_id=1391,
    )
    parks.append(park)

session.add_all(parks)
await session.commit()

In [5]:
from sqlalchemy import String, Integer


session = AsyncSessionMaker()
with open("ev_load_new.json", encoding="utf-8") as file:
    data = json.load(file)
stations = data["stations"]

for s in stations:
    result = await session.execute(
        select(EVStation).filter(
            EVStation.address["Title"].as_string().cast(String) == s["name"]
            if "name" in s
            else EVStation.external_id["id"].as_string().cast(Integer) == s["osm_id"],
            EVStation.city_id == s["city_id"],
        )
    )
    ev = result.scalar()

    values = []
    for w in s["weekdays"].values():
        value_w = 0
        for _, v in w.items():
            value_w += v
        values.append(value_w)
    try:
        ev.use_time = numpy.median(values)
    except:
        print(s["name"])
await session.commit()

EVT 380
ЖК Ол Инклюзив
GREEN DRIVE САНАТОРИЙ БЕЛАРУСЬ СОЧИ


In [3]:
session = AsyncSessionMaker()


session.add_all(
    [
        EVLoad(min_pop=0, max_pop=10000, value=2),
        EVLoad(min_pop=10000, max_pop=100000, value=4),
        EVLoad(min_pop=100000, max_pop=1000000, value=6),
        EVLoad(min_pop=1000000, max_pop=100000000, value=10),
    ]
)
await session.commit()

In [1]:
import json
import numpy

with open("ev_load_new.json", encoding="utf-8") as file:
    data = json.load(file)


stations = data["stations"]

res = {1: [], 2: [], 3: [], 4: []}
for s in stations:
    values = []
    for w in s["weekdays"].values():
        value_w = 0
        for _, v in w.items():
            value_w += v
        values.append(value_w)
    res[s["id"]].append(numpy.median(values))
for id, values in res.items():
    print(id, numpy.median(values))

1 1.0
2 0.6
3 3.499999999999999
4 6.799999999999999


In [3]:
session = AsyncSessionMaker()

result = await session.execute(
        select(EVStation)
    )
for ev in result.scalars():
    ev.use_time=None
await session.commit()


In [2]:
import os
import json
from sqlalchemy_bigquery import WKT
from shapely.geometry import Point
from src.database.models import *
from src.database.base import *

session = AsyncSessionMaker()
root = "dataset_parking"
parkings = []
for item in os.listdir(root):
    path_item = root + "\\" + item
    with open(path_item, encoding="utf-8") as file:
        data = json.load(file)
        city_id = int(item.split(".")[0])
        for parking in data:
            location = WKT(Point(parking["x"], parking["y"]).wkt)
            p = Parking(city_id=city_id, location=location)
            parkings.append(p)
session.add_all(parkings)
await session.commit()

In [5]:
import json
import requests

URL = "http://localhost:8000"
offset = 0
root = "dataset_parking"
parkings = []


response = requests.get(
    f"{URL}/geo/cities/all?limit=100&offset={offset}",
)
response_json = response.json()
for i in response_json['items']:
    city_id_r = i['id']
    for item in os.listdir(root):
        city_id = int(item.split(".")[0])
        if city_id_r==city_id:
            print(i['name'])

Абаза
Абакан
Абатское
Абдулино
Абинск
Абрау-Дюрсо
Августовка
Авдеевка
Авнюгский
Авсюнино
Агалатово
Агидель
Агинское
Агириш
Агой
Аграрное
Агроном
Агроном
Агрыз
Адлер
Адыге-Хабль
Адыгейск
Азнакаево
Азов
Азово
Айкино
Айлино
Айхал
Аксай
Аксарка
Аксубаево
Акташ
Актюбинский
Акуша
Акшуат
Акъяр
Алабайтал
Алабино
Алагир
Алакуртти
Алапаевск
Алатырь
Алгасово
Алгатуй
